In [ ]:
import numpy as np
import torch

In [ ]:
inputs = np.array([[73, 67, 43],
[91, 88, 64],
[87, 134, 58],
[102, 43, 37],
[69, 96, 70]], dtype='float32')

In [ ]:
targets = np.array([[56],
[81],
[119],
[22],
[103]], dtype='float32')

In [ ]:
inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)
print(inputs)
print(targets)

tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.]])
tensor([[ 56.],
        [ 81.],
        [119.],
        [ 22.],
        [103.]])


In [ ]:
w = torch.randn(2, 3, requires_grad=True)
b = torch.randn(2, requires_grad=True)
num_w = w
num_w = num_w.detach().numpy()
print(w)
print(b)

tensor([[ 0.0353,  0.9081,  1.0956],
        [-0.9489,  0.2584,  0.1900]], requires_grad=True)
tensor([2.4532, 1.0516], requires_grad=True)


In [ ]:
def model(x):
    return x @ w.t() + b

In [ ]:
preds = model(inputs)
print(preds)


tensor([[112.9804, -42.7311],
        [155.6921, -50.3936],
        [190.7481, -35.8523],
        [ 85.6377, -77.5896],
        [168.7529, -26.3114]], grad_fn=<AddBackward0>)


In [ ]:
def mse(t1, t2):
    diff = t1 - t2
    print(diff)
    return torch.sum(diff * diff) / diff.numel()
# import torch.nn as nn
# mse_loss = nn.MSELoss()
# output = mse_loss(inputs, targets)
# print(output)
loss=mse(inputs,targets)
print(loss)

tensor([[ 17.,  11., -13.],
        [ 10.,   7., -17.],
        [-32.,  15., -61.],
        [ 80.,  21.,  15.],
        [-34.,  -7., -33.]])
tensor(1023.1334)


In [ ]:
print(w.grad)

None


In [ ]:
preds = model(inputs)
print(preds)

tensor([[-107.9324,  210.9515],
        [-160.0042,  287.0258],
        [-181.0997,  365.3167],
        [ -65.8448,  163.5960],
        [-187.8818,  305.5206]], grad_fn=<AddBackward0>)


In [ ]:
loss = mse(preds, targets)
print(loss)

tensor(45553.8164, grad_fn=<DivBackward0>)


In [ ]:
loss.backward()
print(w.grad)
print(b.grad)

tensor([[-17807.6289, -20821.4375, -12698.2266],
        [ 15981.2158,  17409.8125,  10710.0859]])
tensor([-216.7526,  190.2821])


In [ ]:
with torch.no_grad():
    w -= w.grad * 1e-5
    b -= b.grad * 1e-5
    w.grad.zero_()
    b.grad.zero_()
print(w)

tensor([[ 0.6054, -0.4930, -2.0177],
        [ 0.1009,  1.7438,  1.3742]], requires_grad=True)


In [ ]:
preds = model(inputs)
print(mse(preds, targets))

tensor(31120.2598, grad_fn=<DivBackward0>)


In [ ]:
for i in range(100):
    preds = model(inputs)
    loss = mse(preds, targets)
    loss.backward()
    with torch.no_grad():
        w -= w.grad * 1e-5
        b -= b.grad * 1e-5
        w.grad.zero_()
        b.grad.zero_()

In [ ]:
preds = model(inputs)
loss = mse(preds, targets)
print(loss)

tensor(496.7822, grad_fn=<DivBackward0>)


In [ ]:
preds

tensor([[ 56.],
        [ 81.],
        [119.],
        [ 22.],
        [103.]])

In [ ]:
targets

tensor([[ 56.],
        [ 81.],
        [119.],
        [ 22.],
        [103.]])

Lab4_1

In [ ]:
from torch.utils.data import TensorDataset
train_ds = TensorDataset(inputs, targets)
train_ds[0:3]

(tensor([[ 73.,  67.,  43.],
         [ 91.,  88.,  64.],
         [ 87., 134.,  58.]]), tensor([[ 56.],
         [ 81.],
         [119.]]))

In [ ]:
from torch.utils.data import DataLoader
for xb, yb in train_ds:
    print(xb)
    print(yb)
    break

tensor([73., 67., 43.])
tensor([56.])


In [ ]:
import torch.nn as nn

model = nn.Linear(3, 2)
print(model.weight)
print(model.bias)

Parameter containing:
tensor([[-0.4265,  0.3165,  0.4388],
        [-0.5487, -0.2478, -0.3508]], requires_grad=True)
Parameter containing:
tensor([-0.0286, -0.3085], requires_grad=True)


In [ ]:
list(model.parameters())

[Parameter containing:
 tensor([[ 0.3219, -0.0116, -0.0796],
         [ 0.2999,  0.5169,  0.0361]], requires_grad=True),
 Parameter containing:
 tensor([ 0.1490, -0.3226], requires_grad=True)]

In [ ]:
preds = model(inputs)
preds

tensor([[19.4492, 57.7548],
        [23.3281, 74.7663],
        [21.9849, 97.1284],
        [29.5417, 53.8284],
        [15.6747, 72.5212]], grad_fn=<AddmmBackward0>)

In [ ]:
# mse_loss = nn.MSELoss()
# output = mse_loss(model(inputs), targets)
# print(output)
import torch.nn.functional as F
loss_fn = F.mse_loss
loss = loss_fn(model(inputs), targets)
print(loss)

tensor(16261.8779, grad_fn=<MseLossBackward0>)


<ipython-input-62-afd245e3a75b>:6: UserWarning: Using a target size (torch.Size([5, 1])) that is different to the input size (torch.Size([5, 2])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  loss = loss_fn(model(inputs), targets)


In [ ]:
def fit(num_epochs, model, loss_fn, opt):
  for epoch in range(num_epochs):
    for xb,yb in train_dl:
      # Generate predictions
      pred = model(inputs)
      loss = loss_fn(?,?)
      loss.backward()
      opt.step()
      opt.zero_grad()
print('Training loss: ', loss_fn(model(inputs), targets))

SyntaxError: ignored